In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
n_points = '5k'

if n_points == '5k':
    target_filename = 'target_mesh.vtk'
    source_filename = 'source_mesh.vtk'
elif n_points == '15k':
    target_filename = 'target_mesh_15k.vtk'
    source_filename = 'source_mesh_15k.vtk'
    
target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=False,
                      list_features_to_calc=[], # 'curvatures', min_curvature' 'max_curvature'
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.01,
                      non_rigid_beta=50,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=10,
                      graph_smoothing_iterations=600,
                      feature_smoothing_iterations=1,
                      include_points_as_features=False,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([.1,.1]),
                      n_coords_spectral_ordering=10000,
                      n_coords_spectral_registration=1000,
                      initial_correspondence_type='hungarian',
                      final_correspondence_type='kd')  #'kd' 'hungarian'

Loaded Mesh 1
Beginning Eigen Decomposition
Eigen values are: 
[-3.98708395e-18  8.39246263e-04  1.63007145e-03  2.12549101e-03
  3.13941439e-03  3.77495258e-03  4.01682329e-03]
Computed spectrum 1
Loaded Mesh 2
Beginning Eigen Decomposition
Eigen values are: 
[-9.19459446e-18  8.31236570e-04  1.64152416e-03  2.11362458e-03
  3.09029787e-03  3.88535401e-03  3.92405051e-03]
Computed spectrum 2


Printed statement indicate progress while loading the meshes and computing spectral coordinates. The eigenvalues are printed, and if there were too many null eigenvalues then the eigenvectors will be re-calculated and the eigenvalues re-printed. 


### View mesh features

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [4]:
eig_vec = 2

In [5]:
reg.graph_source.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [6]:
reg.graph_target.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second, if rigid registration was performed first, it will print the rigid registration progress and ultimately the registration "results"/"parameters" 
- Third it will print the non-rigid registration progress. 


In [7]:
reg.align_maps()


Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  0     
     1	|  1     
     2	|  -2    
     3	|  3     
     4	|  -4    
     5	|  5     
*Negative source values means those eigenvectors were flipped*
 
Number of features (including spectral) used for registartion: 3

Non-Rigid (Deformable) Registration Beginning

Iteration:1
ML: -8392.023; 	ML change (error):  8392.023; 	Sigma^2:     0.069; 	Sigma^2 change:     0.105
[                                                                        ]
Iteration:2
ML: -9105.029; 	ML change (error):   713.006; 	Sigma^2:     0.061; 	Sigma^2 change:     0.008
[                                                                        ]
Iteration:3
ML: -9149.170; 	ML change (error):    44.141; 	Sigma^2:     0.053; 	Sigma^2 change:     0.008
[                                                                        ]
Iteration:4
ML: -9211.445; 	ML change (error):    62.275; 	Sigma^2:     0.043; 	Sigm

Iteration:46
ML:-11102.980; 	ML change (error):     0.051; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:47
ML:-11103.027; 	ML change (error):     0.047; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:48
ML:-11103.072; 	ML change (error):     0.044; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:49
ML:-11103.113; 	ML change (error):     0.042; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:50
ML:-11103.153; 	ML change (error):     0.040; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============                                                            ]
Iteration:51
ML:-11103.191; 	ML change (error):     0.038; 	Sigma^2:     0.001; 	Sigma^2 c

Iteration:93
ML:-11104.156; 	ML change (error):     0.013; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================                                                  ]
Iteration:94
ML:-11104.169; 	ML change (error):     0.013; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================                                                  ]
Iteration:95
ML:-11104.182; 	ML change (error):     0.013; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[======================                                                  ]
Iteration:96
ML:-11104.194; 	ML change (error):     0.012; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:97
ML:-11104.206; 	ML change (error):     0.012; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:98
ML:-11104.218; 	ML change (error):     0.012; 	Sigma^2:     0.001; 	Sigma^2 c

Iteration:139
ML:-11104.481; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================                                       ]
Iteration:140
ML:-11104.484; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================                                       ]
Iteration:141
ML:-11104.487; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=================================                                       ]
Iteration:142
ML:-11104.490; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==================================                                      ]
Iteration:143
ML:-11104.493; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[==================================                                      ]
Iteration:144
ML:-11104.495; 	ML change (error):     0.003; 	Sigma^2:     0.001; 	Sig

Iteration:184
ML:-11104.550; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:185
ML:-11104.550; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:186
ML:-11104.551; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:187
ML:-11104.551; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[============================================                            ]
Iteration:188
ML:-11104.552; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=============================================                           ]
Iteration:189
ML:-11104.553; 	ML change (error):     0.001; 	Sigma^2:     0.001; 	Sig

Iteration:233
ML:-11104.564; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=======================================================                 ]
Iteration:234
ML:-11104.564; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[========================================================                ]
Iteration:235
ML:-11104.565; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[========================================================                ]
Iteration:236
ML:-11104.565; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[========================================================                ]
Iteration:237
ML:-11104.565; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[========================================================                ]
Iteration:238
ML:-11104.565; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

Iteration:284
ML:-11104.567; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:285
ML:-11104.567; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:286
ML:-11104.567; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:287
ML:-11104.567; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:288
ML:-11104.567; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sigma^2 change:     0.000
[=====================================================================   ]
Iteration:289
ML:-11104.567; 	ML change (error):     0.000; 	Sigma^2:     0.001; 	Sig

## View the spectral coordinates

In [8]:
print('After registration, the range of coordinate values for:\n Source coordinates:\n'
      '{}\n Target coordinates are: \n{}'.format(np.ptp(reg.source_spectral_coords, axis=0),
                                                         np.ptp(reg.target_spectral_coords, axis=0)))


After registration, the range of coordinate values for:
 Source coordinates:
[1. 1. 1.]
 Target coordinates are: 
[1.00529556 0.97881453 1.0077127 ]


The range of the source are 1.0 because we scaled them to be this. The target are slightly different because they have moved while registering to source coordinates. 

In [9]:
reg.view_aligned_spectral_coords(starting_spectral_coord=0,
                                 include_target_coordinates=True,
                                 include_non_rigid_aligned=True,
                                 include_rigid_aligned=False,
                                 include_unaligned=False,
                                 point_set_colors=None)

Viewer(point_set_colors=array([[0.12156863, 0.46666667, 0.7058824 ],
       [1.        , 0.49803922, 0.0549019…

Above is a 3D view of the spectral (or other) coordinates used for registration. The starting idx for coords can be changed by changing starting_spectral_coord (depending on number of coordinates/features used to register meshes). 

In [10]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Above is the source mesh (geometry 0) and the target mesh (geometry 1). Target is coloured sequentially (0 to n_points). Source is colored according to the target idx that each source point was registered to. Therefore, colours from the target should be in anatomically the same location as the corresponding colour on the source mesh. <br>

Pink is at the same point in the range of values for both meshes. So, it's alignment between them helps affirm the good registration. 

In [11]:
reg.view_aligned_smoothed_spectral_coords()

Viewer(point_set_colors=array([[0.12156863, 0.46666667, 0.7058824 ],
       [1.        , 0.49803922, 0.0549019…

In the above: 
- Orange is the smoothed target points using their graph. 
- Blue is the source coordinates projected onto the target points.

In [12]:
# reg.get_source_mesh_transformed_weighted_avg()
# reg.
reg.set_all_mesh_scalars_to_corresp_target_idx()

Above will create new mesh where the source mesh points are transformed to their single best fitting/closest point on the target mesh. This will likely result in "doubles". The next set of commands averages these positions to the 3 closest points on target to better diffuse them. 

In [13]:
reg.get_average_shape()

Above will create a new mesh that is the average of the two meshes. 

In [14]:
reg.view_meshes(include_target=True,
                include_source=True,
                include_transformed_target=True,
                include_average=True
                )

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Above is overlapping visualization of the four meshes: 
- Original Target
- Original Source
- Transformed Source (to target surface). 
- Average mesh

Possible to turn each one off/on, turn to wire mesh, change transparency, and change colours. 

### Getting Results: 
- After viewing/analyziny results, we can prob the `reg` object to get various results

In [16]:
target_id_per_source = reg.corresponding_target_idx_for_each_source_pt
transformed_source_nn_xyzs = reg.nearest_neighbor_transformed_points
transformed_source_avg_xyzs = reg.weighted_avg_transformed_points
transformed_source_avg_mesh = reg.get_source_mesh_transformed_weighted_avg
transformed_source_nn_mesh = reg.get_source_mesh_transformed_nearest_neighbour
average_mesh = reg.average_mesh

The above shows some of the key metrics/things that can be extracted. Not all will be always be available (depending on what settings were set). 

`target_id_per_source`:           The target mesh cell/node index that corresponds with each source mesh cell/node<br>

`transformed_source_nn_xyzs`:     The transformed xyz positions for each source node to make it aligned with the target mesh while preserving correct anatomical alignment & *using the nearest neighbor for final alignment*.<br>

`transformed_source_avg_xyzs`:    The transformed xyz positions for each source node to make it aligned with the target mesh while preserving correct anatomical alignment & *using the weighted average alignment method*.<br>

`transformed_source_nn_mesh`:     New mesh with nodes transformed using `transformed_source_nn_xyzs`<br>

`transformed_source_avg_mesh`:    New mesh with nodes transformed using `transformed_source_avg_xyzs`<br>

`average_mesh`:                   New mesh calculated as average of `transformed_source_nn_mesh` & `transformed_source_avg_mesh`
